In [ ]:
rom rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Chem import MolStandardize

# syntax edit 1 - no longer openforcefield.topology  
from openff.toolkit import ForceField, Molecule, Topology
from openff.toolkit.topology import Molecule
# syntax edit 2
from openff.toolkit.typing.engines.smirnoff import ForceField as OFFForceField
from openmmforcefields.generators import SystemGenerator
from simtk import openmm, unit
from simtk.openmm import app
from simtk.openmm.app import PDBFile
# syntax edit 3 
from openmm.app.modeller import Modeller 
from simtk.openmm.app import NoCutoff, HBonds



In [ ]:
from pdbfixer import PDBFixer
import pdb4amber
import parmed
import MDAnalysis

In [ ]:


#                                -----   LIGAND PREP  -----



In [ ]:
# syntax edit 4 - -h replaced by -p 7 (to take into account the protonatuon states when H is added)

#adding hydrogens to ligand (ligand.pdb is delinker output)
!obabel ligand.pdb -ipdb -opdb -O ligand_h.pdb -p 7

In [ ]:
#convert to sdf
!obabel ligand_h.pdb -ipdb -osdf -O ligand_h.sdf


In [ ]:
#create pdb object
pdbfile = PDBFile('./ligand_h.pdb')
#create molecule object
mol = Molecule.from_file('./ligand_h.sdf')

# HELPFUL TIP 1 : can view topology at this stage to check by executing: "pdbfile.topology"

In [ ]:
# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[mol])


# Load the OpenFF "Parsley" force field.
forcefield = OFFForceField('openff-1.0.0.offxml') 

# Parametrize the topology and create an OpenMM System.
system = forcefield.create_openmm_system(off_topology)

In [ ]:
omm_topology = pdbfile.getTopology()
positions = pdbfile.getPositions()
# Convert OpenMM System into a ParmEd Structure.
ligand_structure = parmed.openmm.load_topology(omm_topology,
                                                system,
                                                xyz=positions)

In [ ]:


#                                -----   PROTEIN PREP  -----




In [ ]:
#fix PDB
fixer = PDBFixer('./mpro.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
PDBFile.writeFile(fixer.topology, fixer.positions, open('mpro_nosolvent.pdb', 'w'))





In [ ]:
#more PDB fixing + protonation
pdb4amber.run(arg_pdbout='mproh.pdb', arg_pdbin='mpro_nosolvent.pdb', arg_reduce=True, arg_logfile='pdb4amber.log')
!tail -n 20 pdb4amber.log

In [ ]:
#make the system generator, load forcefields
forcefield_kwargs = { 'constraints' : app.HBonds, 'rigidWater' : True, 'removeCMMotion' : False, 'hydrogenMass' : 4*unit.amu }

#system generator used to create system objects
system_gen = SystemGenerator(forcefields=['amber99sbildn.xml', 'tip3pfb.xml'], 
                             small_molecule_forcefield="openff-1.3.0.offxml", 
                             forcefield_kwargs=forcefield_kwargs,
                            molecules=[mol,])

In [ ]:
#make the modeller
pdbfile = PDBFile('mproh.pdb')
top = pdbfile.getTopology()
pos = pdbfile.getPositions()
modeller = app.Modeller(top, pos)

In [ ]:
#now add water 
modeller.addSolvent(system_gen.forcefield, model="tip3p", padding=1, ionicStrength=0*unit.molar, neutralize=False)
top = modeller.topology
pos = modeller.positions

#write a pdb file of solvated protein
with open("mpro_h_solvent.pdb", 'w') as outfile:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

In [ ]:
#parameterize the protein and create a system 
system = system_gen.create_system(topology=modeller.topology)

In [ ]:
#convert the protein system into a ParmEd structure
protein_structure = parmed.openmm.load_topology(top,
                                           system,
                                           xyz=pos)

In [ ]:
#write structure to pdb file
PDBFile.writeFile(protein_structure.topology, 
                  protein_structure.positions, 
                  open('protein_structure.pdb', 'w'))
pdbfile = PDBFile('protein_structure.pdb')

In [2]:


#                                -----   Combining the protein and ligand  -----





In [ ]:
#adding the structures from protein.ipynb and ligand.ipynb
complex_structure = protein_structure + ligand_structure
#writing new structure to a .pdb file
complex_structure.write_pdb('./complex_system.pdb')

In [ ]:


#                                -----   Simulating the protein-ligand comple  -----






In [ ]:
#convert the structure to an openmm system
complex_system = system_gen.create_system(topology=complex_structure.topology)

In [ ]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_min = openmm.LangevinIntegrator(temperature, friction, time_step)

#set up an openmm simulation
simulation = openmm.app.Simulation(complex_structure.topology, complex_system, integrator_min)


#set the initial positions
positions = complex_structure.positions

simulation.context.setPositions(positions)


In [ ]:
simulation.minimizeEnergy()
simulation.saveState('./minimized.state')

In [3]:
# HELPFUL TIP 2 : PRINT OUT MINIMISED COORDINATES AS A PDB - COMPARE WITH ORIGINAL PDB

In [ ]:
#get state of minimised simulation
state = simulation.context.getState(getPositions=True)
#get positions of minimised simulation
positions = state.getPositions()
#write minimised structure to .pdb
PDBFile.writeFile(complex_structure.topology, positions, open("minimised_complex.pdb", "w"))
pdbfile_min = PDBFile('./minimised_complex.pdb')

In [ ]:
# syntax edit 5 

minimised_structure = parmed.openmm.load_topology(minimised_structure.topology, complex_system,xyz=positions)

In [ ]:
#propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator_prod = openmm.LangevinIntegrator(temperature, friction, time_step)

#length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

#set up an OpenMM simulation using minimised structure
simulation = openmm.app.Simulation(minimised_structure.topology, complex_system, integrator_prod)

#set the initial positions.
positions = pdbfile_min.getPositions()
simulation.context.setPositions(positions)


#randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

#configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory_prod.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data_prod.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)



In [ ]:
import time

print("Starting simulation")
start = time.process_time()

#run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")